# ML XGBoost

In [504]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.ensemble import GradientBoostingClassifier as xgb
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [4]:
data = pd.read_csv('./data/SEATGURU_COMPLETE_INFO.csv', sep = ',')
data['plane_seats_info'] = data['plane_seats_info'].apply(lambda x : literal_eval(x))

In [91]:
df = pd.read_csv('./data/SEATGURU_INFO_AIRCRAFT.csv', sep = ';')

In [92]:
df.head()

,Data_Source,Airline_name,Aircraft_Type,Category,Flight_Type,Count,Total_seat,Seat_Class,Seat_Type,Seat_Position,...,Width,Bedlength_maximum,Bedlength_minimum,Music,VideoType,Power_available,Power_kind,Usb_available,Usb_kind,Rating
0,SeatGuru,aegean-airlines,Airbus A319-100,ECONOMY,short haul,138,138,Economy,STANDARD,BULKHEAD,...,18.0,NaN,NaN,NONE,NONE,NONE,NaN,NONE,NaN,2
1,SeatGuru,aegean-airlines,Airbus A319-100,ECONOMY,short haul,138,138,Economy,STANDARD,NORMAL,...,18.0,NaN,NaN,NONE,NONE,NONE,NaN,NONE,NaN,2
2,SeatGuru,aegean-airlines,Airbus A319-100,ECONOMY,short haul,138,138,Economy,STANDARD,EXIT_ROW,...,18.0,NaN,NaN,NONE,NONE,NONE,NaN,NONE,NaN,5
3,SeatGuru,aegean-airlines,Airbus A319-100,ECONOMY,short haul,138,138,Economy,STANDARD,NORMAL,...,18.0,NaN,NaN,NONE,NONE,NONE,NaN,NONE,NaN,3
4,SeatGuru,aegean-airlines,Airbus A319-100,ECONOMY,short haul,138,138,Economy,STANDARD,NORMAL,...,18.0,NaN,NaN,NONE,NONE,NONE,NaN,NONE,NaN,1


In [93]:
df = df.groupby(['Data_Source', 'Airline_name', 'Aircraft_Type', 'Category', 'Seat_Position']).agg(list)

In [94]:
df.head()

Flight_Type  \
Data_Source Airline_name    Aircraft_Type   Category Seat_Position                                         
SeatGuru    aegean-airlines ATR 42-600      ECONOMY  EXIT_ROW                   [short haul, short haul]   
                                                     NORMAL         [short haul, short haul, short haul]   
                            Airbus A319-100 ECONOMY  BULKHEAD                               [short haul]   
                                                     EXIT_ROW                               [short haul]   
                                                     NORMAL         [short haul, short haul, short haul]   

                                                                              Count  \
Data_Source Airline_name    Aircraft_Type   Category Seat_Position                    
SeatGuru    aegean-airlines ATR 42-600      ECONOMY  EXIT_ROW              [48, 48]   
                                                     NORMAL            [48, 48, 48]   
                            Airbus A319-100 ECONOMY  BULKHEAD                 [138]   
                                                     EXIT_ROW                 [138]   
                                                     NORMAL         [138, 138, 138]   

                                                                         Total_seat  \
Data_Source Airline_name    Aircraft_Type   Category Seat_Position                    
SeatGuru    aegean-airlines ATR 42-600      ECONOMY  EXIT_ROW              [48, 48]   
                                                     NORMAL            [48, 48, 48]   
                            Airbus A319-100 ECONOMY  BULKHEAD                 [138]   
                                                     EXIT_ROW                 [138]   
                                                     NORMAL         [138, 138, 138]   

                                                                                     Seat_Class  \
Data_Source Airline_name    Aircraft_Type   Category Seat_Position                                
SeatGuru    aegean-airlines ATR 42-600      ECONOMY  EXIT_ROW                [Economy, Economy]   
                                                     NORMAL         [Economy, Economy, Economy]   
                            Airbus A319-100 ECONOMY  BULKHEAD                         [Economy]   
                                                     EXIT_ROW                         [Economy]   
                                                     NORMAL         [Economy, Economy, Economy]   

                                                                                         Seat_Type  \
Data_Source Airline_name    Aircraft_Type   Category Seat_Position                                   
SeatGuru    aegean-airlines ATR 42-600      ECONOMY  EXIT_ROW                 [STANDARD, STANDARD]   
                                                     NORMAL         [STANDARD, STANDARD, STANDARD]   
                            Airbus A319-100 ECONOMY  BULKHEAD                           [STANDARD]   
                                                     EXIT_ROW                           [STANDARD]   
                                                     NORMAL         [STANDARD, STANDARD, STANDARD]   

                                                                            Recline  \
Data_Source Airline_name    Aircraft_Type   Category Seat_Position                    
SeatGuru    aegean-airlines ATR 42-600      ECONOMY  EXIT_ROW            [nan, nan]   
                                                     NORMAL         [nan, nan, nan]   
                            Airbus A319-100 ECONOMY  BULKHEAD                 [nan]   
                                                     EXIT_ROW                 [nan]   
                                                     NORMAL         [nan, nan, nan]   

                                                                      Premium  \
Data_Source Airl

In [475]:
df_image = pd.read_csv('./data/Images.csv')
df_image.columns = ['name_image', 'lines', 'elements', 'caracteres', 'same_nb_seats', 'sorties']

In [476]:
df_image['elements'] = df_image['elements'].apply(lambda x: x[:-1].split(';'))

In [477]:
def decompose(list_elements):
    list_col = []
    list_row = []
    for value in list_elements:
        col, row = value.split(':')
        list_col.append(col)
        list_row.append(row)
    return list_col, list_row

In [478]:
df_image['elements'] = df_image['elements'].apply(lambda x: decompose(x))

In [479]:
list_col = []
for row in df_image['elements']:
    l = row[0]
    for j in l:
        list_col.append(j)
df_final = pd.DataFrame(columns = pd.Series(list_col).unique())

In [480]:
def concat_df(liste, df):
    to_concat = pd.DataFrame(np.array([liste[1]]), columns = liste[0])
    return pd.concat([df, to_concat], ignore_index = 
                    True, sort = False)
#     return pd.DataFrame(np.array([liste[1]]), columns = liste[0])
#     print(df)
#     pd.concat([df_base, df], axis = 'columns')
#     return df_base

In [481]:
for liste in df_image['elements']:
    df_final = concat_df(liste, df_final)

In [482]:
df_image = df_image.join(df_final)

In [483]:
df_image.head()

,name_image,lines,elements,caracteres,same_nb_seats,sorties,siege,toilette,bar,exit,...,residence,inconnu,cintre,carre,toilette handicape,fauteil,verre,fauteuil,vide,telephone
0,easyjet_airlines_airbus_A319_plane354,4016-4023,"([siege, toilette, bar, exit], [156, 3, 1, 8])",n,y,avant / milieu / arriere,156,3,1,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,easyjet_airlines_airbus_A320_plane355,4024-4030,"([siege, toilette, bar, exit], [180, 3, 2, 8])",n,y,avant / milieu / arriere,180,3,2,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,easyjet_airlines_airbus_A320_V2_plane356,4031-4036,"([siege, toilette, bar, exit], [186, 3, 1, 8])",n,y,avant / milieu / arriere,186,3,1,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,edelweeiss_air_A320-200_plane359,4037-4044,"([siege, toilette, bar, exit], [216, 3, 2, 8])",n,y,avant / milieu / arriere,216,3,2,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,edelweiss_air_a330-300_plane357,4045-4053,"([siege, toilette, bar, exit], [168, 7, 3, 8])",n,n,avant / milieu / arriere,168,7,3,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [484]:
df_image = df_image.fillna(0)

In [485]:
for col in ['siege', 'toilette', 'bar', 'exit', 'cafe', 'restaurant',
       'toilette handicape', ' cafe', 'salle de jeu', 'residence', 'inconnu',
       'cintre', 'carre', 'toilette handicape ', 'fauteil', 'verre',
       'fauteuil', 'vide', 'telephone']:
    df_image[col].astype(int)

In [486]:
df_image['bar'] = df_image.apply(lambda x: int(x['bar']) + int(x['cafe']) + int(x[' cafe']), axis = 1)
df_image.drop(' cafe', inplace = True, axis = 1)
df_image.drop('cafe', inplace = True, axis = 1)
df_image['toilette handicape'] = df_image.apply(lambda x: int(x['toilette handicape']) + int(x['toilette handicape ']), axis = 1)
df_image.drop('toilette handicape ', inplace = True, axis = 1)
df_image['fauteuil'] = df_image.apply(lambda x: int(x['fauteil']) + int(x['fauteuil']), axis = 1)
df_image.drop('fauteil', inplace = True, axis = 1)

In [487]:
df_image.columns

Index(['name_image', 'lines', 'elements', 'caracteres', 'same_nb_seats',
       'sorties', 'siege', 'toilette', 'bar', 'exit', 'restaurant',
       'toilette handicape', 'salle de jeu', 'residence', 'inconnu', 'cintre',
       'carre', 'verre', 'fauteuil', 'vide', 'telephone'],
      dtype='object')

In [505]:
# X = df_image[['toilette', 'bar', 'exit', 'restaurant',
#        'toilette handicape', 'salle de jeu', 'residence', 'inconnu', 'cintre',
#        'carre', 'verre', 'fauteuil', 'vide', 'telephone']].astype(int)
X = df_image[['toilette', 'bar', 'exit', 'restaurant']].astype(int)
y = df_image[['siege']].astype(int)

In [506]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [507]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [508]:
xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [509]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 88.469742


In [510]:
preds

array([148.32831 , 113.758095, 100.615166, 122.27831 , 264.68723 ,
       109.40885 , 109.40885 , 122.27831 , 236.69473 ,  69.011154,
       256.05325 , 109.40885 , 100.615166, 193.54126 , 180.03955 ,
        66.21919 ,  89.80779 , 204.20616 , 290.71292 , 228.95386 ,
       204.20616 , 122.27831 ,  69.011154,  47.61287 , 115.902306,
       122.27831 , 214.71019 , 226.33418 , 204.20616 , 122.27831 ,
       184.02121 , 188.93741 , 173.19626 , 298.18393 , 214.71019 ,
        85.76597 ,  98.209435, 115.902306, 115.902306, 132.12122 ,
       204.20616 , 193.54126 , 236.69473 , 115.902306, 184.02121 ,
       122.27831 , 264.68723 , 109.40885 ,  24.074512, 226.33418 ,
       226.33418 , 115.902306, 188.93741 ,  66.21919 ,  81.73191 ,
       102.34175 ,  95.395515, 236.69473 ,  89.80779 , 115.902306,
       170.11655 , 105.392944,  95.395515, 175.96234 , 175.96234 ,
        91.720024, 173.19626 ,  49.52302 ,  49.52302 , 167.3024  ,
       109.40885 , 109.40885 ], dtype=float32)